In [5]:
from sklearn.cluster import KMeans
import pandas as pd
import pickle
import string
import re
import string
from sklearn.model_selection import cross_validate as cross_validation, ShuffleSplit, cross_val_score, train_test_split, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, auc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pickle

In [14]:
data_pre = pd.read_pickle('Cyclical_encoded_data.pkl')

In [15]:
imp = IterativeImputer(max_iter=10, random_state=0)
data = data_pre.drop(['RHoMIS_ID','ID_HH'],axis=1)
imp.fit_transform(data)

array([[ 2.019e+03,  1.000e+00,  1.119e+01, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 2.019e+03,  1.000e+00,  1.119e+01, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 2.019e+03,  1.000e+00,  1.119e+01, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       ...,
       [ 2.017e+03,  1.000e+00, -1.382e+01, ...,  0.000e+00,  0.000e+00,
         1.000e+00],
       [ 2.017e+03,  1.000e+00, -1.382e+01, ...,  0.000e+00,  0.000e+00,
         1.000e+00],
       [ 2.017e+03,  1.000e+00, -1.382e+01, ...,  0.000e+00,  0.000e+00,
         1.000e+00]])

In [16]:
data.to_pickle('imputed_data.pkl')